In [ ]:
#ojo espetrograma y escalogramacwt uso la misma escalapara cada audio

# trabajo completo de analisis de audios

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as waves
import glob
import pandas as pd
from sklearn.decomposition import PCA
import Ipython.display as ipd
import scipy.fftpack as fourier

### formación de la matriz



In [ ]:
wav_files = glob.glob('C:\Users\migue\OneDrive\Escritorio\Semestre 7\Metodos de machine learning\semana 2 a 4 audios y espectros\Audios Machine Learning\*.wav') #importando cada nombre 
wav_files.remove('9.wav')
wav_files.remove('14.wav')# se removieron los audios 14 y 9
print(f'lista de audios cargados:{wav_files}')

mat=[]#matriz con cada una de las filas un audio


for j in range(0,len(wav_files)):
    print(wav_files[j])
    srate,data=waves.read(wav_files[j]) # importamos el audio y formamos un vector de este, usando el canal 1 del audio en estéreo se recortara.
    length = data.shape[0] / srate
    time = np.linspace(0., length, data.shape[0])
    #print(f'dimension de audio original {data.shape[0]}')

    data1=data[:,1][43000:172000] # recortamos el vector de audio obtenido debido a que al inicio y al final de este el sonido es ruido o silencio.
    length1 =len(data1)/ srate    # el audio se recorta 1/4 tanto al inicio como al final 
    time1 = np.linspace((43000)/srate, length1, data1.shape[0])
    #print(f'dimension de audio cortado {data1.shape[0]}')

    print('Pre-proceso realizado')

    mat.append(data1)
    '''# cuadro comentado de las graficas de cada uno de los audios para poder comparar el original y el recortado
    plt.plot(time, data[:, 1], label="orginal")
    plt.plot(time1+0.578, data1,  label="cortado")

    plt.legend()
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.show()
    '''


print('--------------------------------------------------------------------------')
matrix=np.stack( mat, axis=0 ) #formación de la matriz cuyas filas son los archivos de audio

#for k in range(0,len(mat)): #verificar dimensiones de los vectores
    #print(len(mat[k]))

print(matrix)

lista de audios cargados:['1.wav', '10.wav', '11.wav', '12.wav', '13.wav', '15.wav', '16.wav', '17.wav', '18.wav', '19.wav', '2.wav', '20.wav', '21.wav', '22.wav', '23.wav', '24.wav', '25.wav', '26.wav', '27.wav', '28.wav', '29.wav', '3.wav', '30.wav', '31.wav', '32.wav', '33.wav', '34.wav', '4.wav', '5.wav', '6.wav', '7.wav', '8.wav']
1.wav
dimension de audio original 222720
dimension de audio cortado 129000
-----------------
10.wav
dimension de audio original 222720
dimension de audio cortado 129000
-----------------
11.wav
dimension de audio original 207360
dimension de audio cortado 129000
-----------------
12.wav
dimension de audio original 222720
dimension de audio cortado 129000
-----------------
13.wav
dimension de audio original 230400
dimension de audio cortado 129000
-----------------
15.wav
dimension de audio original 192000
dimension de audio cortado 129000
-----------------
16.wav
dimension de audio original 207360
dimension de audio cortado 129000
-----------------
17.wa

### Analisis de valores singulares

In [ ]:
U,S,VT=np.linalg.svd(matrix, full_matrices=False)
fig = plt.figure(figsize=(17,6))
ax1=fig.add_subplot(121)
ax1.set_title('logaritmo de los valores singulares')
ax1.semilogy(S,'-o',color='k')
ax2 = fig.add_subplot(122)
ax2.set_title('porcentaje de información  por números de valores singulares')
ax2.plot(np.cumsum(S)/np.sum(S),'-o',color='b')

### Descomposicion en componentes principales

usar funcion de normalizacionç

crear etiquetas